In [36]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

Lyrics classifier. Trying to get the mood out of the song. 

Import the MasterSong.json file first...

In [4]:
import pandas as pd
import numpy as np

In [5]:
songs_df = pd.read_json('MasterSongList.json')

In [6]:
songs_lf_m = ['lyrics_features', 'moods']
lyrics_df = songs_df.copy()
lyrics_df = lyrics_df[songs_lf_m]
lyrics_df.head()

,lyrics_features,moods
0,"[oppa, gangnam, style, gangnam, style, najeneu...","[energetic, motivational]"
1,"[lately, i, ve, been, i, ve, been, losing, sle...",[happy]
2,"[party, rock, yeah, woo, let, s, go, party, ro...","[happy, celebratory, rowdy]"
3,"[alagamun, lan, weh, wakun, heya, hanun, gon, ...","[happy, energetic, celebratory]"
4,"[j, lo, the, other, side, out, my, mine, it, s...",[energetic]


In [7]:
lyrics_df['lyrics_features'] = lyrics_df['lyrics_features'].apply(' '.join)
lyrics_df['moods'] = lyrics_df['moods'].apply(', '.join)

In [9]:
lyrics_df.head()

,lyrics_features,moods
0,oppa gangnam style gangnam style najeneun ttas...,"energetic, motivational"
1,lately i ve been i ve been losing sleep dreami...,happy
2,party rock yeah woo let s go party rock is in ...,"happy, celebratory, rowdy"
3,alagamun lan weh wakun heya hanun gon alagamun...,"happy, energetic, celebratory"
4,j lo the other side out my mine it s a new gen...,energetic


In [13]:
#NaN songs.
lyrics_df['lyrics_features'].replace('', np.nan, inplace=True)

In [14]:
lyrics_df.shape

(36733, 2)

In [15]:
lyrics_df.dropna(subset=['lyrics_features'], inplace=True)

In [16]:
lyrics_df.shape

(20931, 2)

In [17]:
#Reindex dataframe
lyrics_df.reset_index(drop=True, inplace=True)

Lyrics classifier. Trying to get the mood out of the song. 
First I must clean the data...

In [1]:
from string import punctuation
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.stem.snowball import SnowballStemmer

1. Converting everything to lower case
2. Removing punctuation
3. Removing common words (stop words)
4. Stemming

In [2]:
def clean_text(raw_text):
    # Create empty list to receive result
    clean_words = []
    
    # 1. Convert to lower case
    raw_text = raw_text.lower()
    
    # 2. Remove punctuation
    translator = str.maketrans('', '', punctuation)
    raw_text = raw_text.translate(translator)
    split_words = raw_text.split()
    
    # 3 & 4. Remove common words and stem words
    stemmer = SnowballStemmer('english')
    for word in split_words:
        if word not in ENGLISH_STOP_WORDS:
            stemmed_word = stemmer.stem(word)
            clean_words.append(stemmed_word)
            
    return ' '.join(clean_words)

In [30]:

songs_df['lyrics_features']=[" ".join(lyrics_features) for lyrics_features in songs_df['lyrics_features'].values]

In [24]:
moods = songs_df['moods'].tolist()
moods_set = set(x for i in moods for x in i)
moods_set

{'aggressive',
 'angsty',
 'atmospheric',
 'campy',
 'celebratory',
 'classy',
 'cocky',
 'cold',
 'earthy',
 'energetic',
 'funky',
 'gloomy',
 'happy',
 'hypnotic',
 'introspective',
 'lush',
 'mellow',
 'motivational',
 'nocturnal',
 'raw',
 'rowdy',
 'sad',
 'seductive',
 'sexual',
 'soothing',
 'spacey',
 'sprightly',
 'sweet',
 'trashy',
 'trippy',
 'visceral',
 'warm'}

# Random forest classifier bag of words ?

In [26]:
count_vect = CountVectorizer()

In [29]:
songs_df['lyrics_features']

0        [oppa, gangnam, style, gangnam, style, najeneu...
1        [lately, i, ve, been, i, ve, been, losing, sle...
2        [party, rock, yeah, woo, let, s, go, party, ro...
3        [alagamun, lan, weh, wakun, heya, hanun, gon, ...
4        [j, lo, the, other, side, out, my, mine, it, s...
5        [today, i, don, t, feel, like, doing, anything...
6        [there, s, a, fire, starting, in, my, heart, r...
7        [i, threw, a, wish, in, the, well, don, t, ask...
8        [now, and, then, i, think, of, when, we, were,...
9        [don, t, know, what, for, you, re, turning, he...
10       [nossa, nossa, assim, você, me, mata, ai, se, ...
11       [shine, bright, like, a, diamond, shine, brigh...
12       [do, you, ever, feel, like, a, plastic, bag, d...
13       [oh, oh, woah, oh, oh, oh, oh, oh, oh, oh, oh,...
14       [girl, my, body, don, t, lie, red, one, i, m, ...
15       [ohh, ohh, ohh, ohh, ohh, oh, her, eyes, her, ...
16       [it, s, our, party, we, can, do, what, we, wan.

In [31]:
bag_of_words = count_vect.fit_transform(songs_df['lyrics_features'])

In [38]:
moods = songs_df['moods']

In [41]:
X = bag_of_words
y = moods

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [42]:
rfc = RandomForestClassifier(n_estimators=5, min_samples_split=2, max_features='log2')
rfc.fit(X_train, y_train)
rfc_predictions = rfc.predict(X_test)
print(accuracy_score(y_test, rfc_predictions))

ValueError: Unknown label type: 'unknown'